In [1]:
# # The Base Version

# import os
# import re
# from flask import Flask, request
# from slack_sdk import WebClient
# from slack_bolt import App
# from slack_bolt.adapter.flask import SlackRequestHandler
# import pandas as pd
# import fitz  # PyMuPDF
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores import FAISS
# from langchain.chains import LLMChain
# from langchain.chains.question_answering import load_qa_chain
# from langchain.llms import OpenAI
# from langchain.chat_models import ChatOpenAI
# from langchain.memory import ConversationBufferMemory
# from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
# from dotenv import find_dotenv, load_dotenv

# # Load environment variables
# load_dotenv(find_dotenv())

# # Set Slack API credentials
# SLACK_BOT_TOKEN = os.environ["SLACK_BOT_TOKEN"]
# SLACK_SIGNING_SECRET = os.environ["SLACK_SIGNING_SECRET"]

# # Initialize the Slack app and Flask app only once
# app = App(token=SLACK_BOT_TOKEN, signing_secret=SLACK_SIGNING_SECRET)  # Corrected initialization
# flask_app = Flask(__name__)
# handler = SlackRequestHandler(app)

# # OpenAI-based language model with conversational capabilities
# chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)

# # Create a memory buffer for maintaining context in conversations
# # memory = ConversationBufferMemory()
# memory = ConversationBufferMemory(memory_key="conversation", return_messages=True, k=20)

# # Create system and human prompts
# system_template =system_template = """
# You are Alphie, a highly capable AI assistant.
# Your main roles are to:
# - Answer user questions with detailed and accurate information.
# - Assist with various tasks including programming, debugging, writing, data analysis, and more.
# - Provide suggestions for personal and professional development.
# - Help with productivity, organization, and automation.
# - Engage in meaningful conversations, maintaining context to ensure continuity.

# Key characteristics:
# - Be friendly, helpful, and responsive.
# - Offer clear explanations and useful advice.
# - When necessary, reference sources to increase reliability and transparency.
# - Guide users to the appropriate resources or solutions, and be clear about any limitations.

# Specific guidance:
# - For technical queries, aim to be thorough and offer code examples when appropriate.
# - For general knowledge, be informative and concise.
# - For questions about specific documents or content, extract relevant information and mention sources.
# - When unsure, ask users for clarification or suggest additional resources.

# Your mission is to create a seamless user experience, helping users achieve their goals while ensuring a clear and supportive communication style.
# """


# system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
# human_template = "User says: {user_input}."
# human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# # Conversational chain with memory
# chain = LLMChain(llm=chat, prompt=chat_prompt, memory=memory)

# # Function to extract text from PDF
# def extract_text_from_pdf(pdf_path):
#     reader = PdfReader(pdf_path)
#     raw_text = ""
#     for page in reader.pages:
#         text = page.extract_text()
#         if text:
#             raw_text += text + "\n"
#     return raw_text

# # Function to clean text
# def clean_text(text):
#     text = re.sub(r'\s+', ' ', text)  # Replace multiple whitespace with single space
#     text = text.strip()  # Strip leading and trailing whitespace
#     return text

# # Function to analyze PDF content
# def analyze_pdf(pdf_path, question):
#     raw_text = extract_text_from_pdf(pdf_path)
#     cleaned_text = clean_text(raw_text)

#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=1500,
#         chunk_overlap=400,
#         separators=["\n", " ", "."],  # Better context splitting
#     )
#     texts = text_splitter.split_text(cleaned_text)

#     embeddings = OpenAIEmbeddings()
#     docsearch = FAISS.from_texts(texts, embeddings)

#     qa_chain = load_qa_chain(OpenAI(), chain_type="refine")

#     docs = docsearch.similarity_search(question, k=7)  # Context for similarity search
#     answer = qa_chain.run(input_documents=docs, question=question)

#     return answer

# # Load the mapping from the CSV file for keyword-to-PDF mapping
# csv_path = "C:/Users/asifr\OneDrive - State of New Mexico\Documents/GitHub/langchain-experiments/slack/output_tables/article_titles.csv"  # Update with the correct path
# pdf_dir = "articles_output"  # Directory with PDFs

# article_pdf_map = {}
# df = pd.read_csv(csv_path)

# # Populate the dictionary with ARTICLE numbers and short titles
# for _, row in df.iterrows():
#     article_number = row["ARTICLE Number"]
#     short_title = row["Short Title"]

#     pdf_name = f"ARTICLE {article_number}.pdf"
#     pdf_path = os.path.join(pdf_dir, pdf_name)

#     article_pdf_map[article_number] = pdf_path
#     article_pdf_map[short_title.lower()] = pdf_path

# # Function to get the PDF path based on a keyword
# def get_pdf_path_from_keyword(keyword):
#     keyword = keyword.lower()
#     for key in article_pdf_map:
#         if keyword in key:
#             return article_pdf_map[key]
#     return None

# # Slack event handler for message events
# @app.event("message")
# def handle_message_events(event, say):
#     user_text = event.get("text", "").lower()
#     channel_type = event.get("channel_type", "")

#     # Respond to greetings
#     if user_text in ["hi", "hello", "hey"]:
#         say("Hello! I'm Alphie, your friendly AI assistant at the New Mexico Tax & Rev. How can I help you today?")
#         return

#     # Check if the message contains a keyword that maps to a PDF
#     pdf_path = get_pdf_path_from_keyword(user_text)

#     if pdf_path:
#         question = re.sub(r"(" + "|".join(article_pdf_map.keys()) + ")", "", user_text).strip()
        
#         response = analyze_pdf(pdf_path, question)

#         if not response or "I don't know" in response:
#             say("I'm not sure I understood that. Could you please rephrase your question?")
#         else:
#             say(response)
#         return

#     # Default response for other queries
#     response = chain.run({"user_input": user_text})
#     say(response)

# # Flask endpoint for Slack events
# @flask_app.route("/slack/events", methods=["POST"])
# def slack_events():
#     return handler.handle(request)

# # # Initialize the Flask app
# if __name__ == "__main__":
#     flask_app.run(port=8080)


In [ ]:
import os
import re
from flask import Flask, request
from slack_sdk import WebClient
from slack_bolt import App
from slack_bolt.adapter.flask import SlackRequestHandler
import pandas as pd
import fitz  # PyMuPDF
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from dotenv import find_dotenv, load_dotenv
from serpapi import GoogleSearch
import wikipedia

# Load environment variables
load_dotenv(find_dotenv())

# Set Slack API credentials
SLACK_BOT_TOKEN = os.environ["SLACK_BOT_TOKEN"]
SLACK_SIGNING_SECRET = os.environ["SLACK_SIGNING_SECRET"]

# Initialize the Slack app and Flask app only once
app = App(token=SLACK_BOT_TOKEN, signing_secret=SLACK_SIGNING_SECRET)
flask_app = Flask(__name__)
handler = SlackRequestHandler(app)

# OpenAI-based language model with conversational capabilities
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)

# Create a memory buffer for maintaining context in conversations
memory = ConversationBufferMemory(memory_key="conversation", return_messages=True, k=20)

# Create system and human prompts
system_template = """
You are Alphie, a highly capable AI assistant.
Your main roles are to:
- Answer user questions with detailed and accurate information.
- Assist with various tasks, including programming, debugging, writing, data analysis, and more.
- Provide suggestions for personal and professional development.
- Help with productivity, organization, and automation.
- Engage in meaningful conversations, maintaining context to ensure continuity.

Key characteristics:
- Be friendly, helpful, and responsive.
- Offer clear explanations and useful advice.
- When necessary, reference sources to increase reliability and transparency.
- Guide users to appropriate resources or solutions, and be clear about any limitations.

Specific guidance:
- For technical queries, aim to be thorough and offer code examples when appropriate.
- For general knowledge, be informative and concise.
- For questions about specific documents or content, extract relevant information and mention sources.
- When unsure, ask users for clarification or suggest additional resources.

Your mission is to create a seamless user experience, helping users achieve their goals while ensuring a clear and supportive communication style.
"""

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template = "User says: {user_input}."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# Conversational chain with memory
chain = LLMChain(llm=chat, prompt=chat_prompt, memory=memory)

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    reader = fitz.open(pdf_path)
    raw_text = ""
    for page in reader:
        text = page.get_text()
        if text:
            raw_text += text + "\n"
    return raw_text

# Function to clean text
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Replace multiple whitespace with a single space
    text = text.strip()  # Strip leading and trailing whitespace
    return text

# Function to analyze PDF content
def analyze_pdf(pdf_path, question):
    raw_text = extract_text_from_pdf(pdf_path)
    cleaned_text = clean_text(raw_text)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=400,
        separators=["\n", " ", "."],
    )
    texts = text_splitter.split_text(cleaned_text)

    embeddings = OpenAIEmbeddings()
    docsearch = FAISS.from_texts(texts, embeddings)

    qa_chain = load_qa_chain(OpenAI(), chain_type="refine")

    docs = docsearch.similarity_search(question, k=7)  # Context for similarity search
    answer = qa_chain.run(input_documents=docs, question=question)

    return answer

# Load the mapping from the CSV file for keyword-to-PDF mapping
csv_path = "C:/Users/asifr/OneDrive - State of New Mexico\Documents/GitHub/langchain-experiments/slack/output_tables/article_titles.csv"  # Update with correct path
pdf_dir = "articles_output"  # Directory with PDFs

article_pdf_map = {}
df = pd.read_csv(csv_path)

# Populate the dictionary with ARTICLE numbers and short titles
for _, row in df.iterrows():
    article_number = row["ARTICLE Number"]
    short_title = row["Short Title"]

    pdf_name = f"ARTICLE {article_number}.pdf"
    pdf_path = os.path.join(pdf_dir, pdf_name)

    article_pdf_map[article_number] = pdf_path
    article_pdf_map[short_title.lower()] = pdf_path

# Function to get the PDF path based on a keyword
def get_pdf_path_from_keyword(keyword):
    keyword = keyword.lower()
    for key in article_pdf_map:
        if keyword in key:
            return article_pdf_map[key]
    return None

# Function to perform Google Search
def google_search(query):
    API_KEY = os.environ["GOOGLE_SEARCH_API_KEY"]
    params = {
        "q": query,
        "api_key": API_KEY,
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    return results

# Function to get a Wikipedia summary
def get_wikipedia_summary(topic):
    try:
        summary = wikipedia.summary(topic, sentences=2)
        return summary
    except wikipedia.exceptions.PageError:
        return "I couldn't find any information on that topic in Wikipedia."
    except wikipedia.exceptions.DisambiguationError:
        return "This topic is ambiguous. Could you be more specific?"

# Slack event handler for message events
@app.event("message")
def handle_message_events(event, say):
    user_text = event.get("text", "").lower()

    # Respond to greetings
    if user_text in ["hi", "hello", "hey"]:
        say("Hello! I'm Alphie, your friendly AI assistant at the New Mexico Tax & Rev. How can I help you today?")
        return

    # Google Search functionality
    if "search" in user_text:
        query = user_text.replace("search", "").strip()
        search_results = google_search(query)

        results_text = "\n".join(
            [f"- {result['title']}: {result['link']}" for result in search_results.get("organic_results", [])[:3]]
        )
        say(f"Here are some search results for '{query}':\n{results_text}")
        return

    # Wikipedia search functionality
    if "what is" in user_text:
        topic = user_text.replace("what is", "").strip()
        wikipedia_summary = get_wikipedia_summary(topic)
        say(wikipedia_summary)
        return

    # Default response for other queries
    if pdf_path := get_pdf_path_from_keyword(user_text):
        question = re.sub(r"(" + "|".join(article_pdf_map.keys()) + ")", "", user_text).strip()
        response = analyze_pdf(pdf_path, question)

        if not response or "I don't know" in response:
            say("I'm not sure I understood that. Could you please rephrase your question?")
        else:
            say(response)
    else:
        response = chain.run({"user_input": user_text})
        say(response)

# Flask endpoint for Slack events
@flask_app.route("/slack/events", methods=["POST"])
def slack_events():
    return handler.handle(request)

# Initialize the Flask app
if __name__ in "__main__":
    flask_app.run(port=8080)


C:\Users\asifr\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


 * Serving Flask app '__main__'
 * Debug mode: off


C:\Users\asifr\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
 * Running on http://127.0.0.1:8080
Press CTRL+C to quit
127.0.0.1 - - [11/May/2024 00:23:24] "POST /slack/events HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2024 00:23:25] "POST /slack/events HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2024 01:04:47] "POST /slack/events HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2024 01:04:48] "POST /slack/events HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2024 01:10:06] "POST /slack/events HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2024 01:10:07] "POST /slack/events HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2024 01:10:44] "POST /slack/events HTTP/1.1" 200 -
C:\Users\asifr\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` 